In [1]:
import tensorflow as tf
import numpy as np
import argparse
import IPython.display as display
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
import copy as cp
from sklearn.model_selection import KFold
from IRM_methods import *

D:\Users\charm\anaconda3\envs\deepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\charm\anaconda3\envs\deepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\charm\anaconda3\envs\deepLearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\charm\anaconda3\envs\deepLearning\lib\site-

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
n_e = 3  # number of environments

In [4]:
num_classes = 5

In [5]:
# List of models - last one is the representation (common)
model_list = []
for e in range(n_e):
    if(e<n_e-1): 
        model_list.append( keras.Sequential([
            keras.layers.Flatten(input_shape=(390,1)),
            keras.layers.Dense(390, activation = 'relu',kernel_regularizer=keras.regularizers.l2(0.00125)),
            keras.layers.Dropout(0.75),
            keras.layers.Dense(390, activation='relu',kernel_regularizer=keras.regularizers.l2(0.00125)),
            keras.layers.Dropout(0.75),
            keras.layers.Dense(num_classes)
        ]))
    if(e==n_e-1):
        model_list.append(keras.Sequential([
        
        keras.layers.Dense(390,input_shape=(400,), activation = 'relu',kernel_regularizer=keras.regularizers.l2(0.00125)),
      ]))

In [6]:
# Hyperparameters
num_epochs       = 5
batch_size       = 256
termination_acc  = 0.6
warm_start       = 100
learning_rate    = 2.5e-4

In [7]:
c = []
for i in range(1, n_e+1):
    
    X = np.load('X' + str(i)+'.npy')
    y = np.load('y' + str(i)+'.npy')
    
    c.append((X,y))
    

In [8]:
# create the game
V_game = variable_irm_game_model(model_list, learning_rate, num_epochs, batch_size, termination_acc, warm_start) 

In [ ]:
# train
V_game.fit(c)

starting training
Epoch: 0
Epoch: 1


In [ ]:
# load data for evaluation
X = np.load('X4.npy')
y = np.load('y4.npy')

c_eval = [(X,y)]

In [ ]:
V_game.evaluate(c_eval) 

# print train and test accuracy
print ("Training accuracy " + str(V_game.train_acc)) 
print ("Testing accuracy " + str(V_game.test_acc))